## Importar bibliotecas

In [98]:
import os
import re
import csv
from datetime import datetime
import pytz
from shapely.geometry import shape, Point
import geopandas as gpd
from geopy.distance import distance
import pandas as pd
from h3 import h3
from geopy.distance import geodesic
import shapefile
from shapely.geometry import Point, shape
from shapely.ops import unary_union
import findspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import col, lit, concat_ws, collect_set
from pyspark.sql.types import TimestampType, StructField, StringType, IntegerType, StructType
from pyspark.sql.functions import date_format
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# from pyspark.ml.regression import LinearRegression
from sklearn.linear_model import LinearRegression
from pyspark.sql.functions import lit
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.sql import functions as F
import json

## Obtener rutas de los Datasets

In [ ]:
# Obtener la ruta de la carpeta del notebook
notebook_folder = os.getcwd()
root_project = os.path.abspath(os.path.join(notebook_folder, '..'))
dataset_logs = os.path.abspath(os.path.join(root_project, 'Datos', 'Logs'))
trufi_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App'))
municipios_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos de Cochabamba','region_cochabamba_2018.geojson'))
lagos_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos de Cochabamba','region_cochabamba_2018.geojson'))
clima_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Datos del clima','weather.csv'))
lagos_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Poligonos de Cochabamba','lagos.shx'))
h3_datos = os.path.abspath(os.path.join(root_project, 'Datos', 'Registros de Trufi App','id_index_h3.csv'))

## Extraer solicitudes de logs a origen-destino.csv

In [87]:
# Definir la zona horaria de La Paz, Bolivia
la_paz_timezone = pytz.timezone('America/La_Paz')
def extract_route_info(log_line):
    # Expresión regular para extraer información específica de las solicitudes de rutas
    route_pattern_with_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+).*?Trufi/.*?/([a-f0-9-]+)')
    route_pattern_without_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+)')

    match_with_id = route_pattern_with_id.search(log_line)
    match_without_id = route_pattern_without_id.search(log_line)

    # Inicializar variables con valores predeterminados
    origin_latitude = origin_longitude = dest_latitude = dest_longitude = id_user = None

    try:
        if match_with_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = match_with_id.groups()
        elif match_without_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude = match_without_id.groups()
            id_user = 'N/A'  # Asignar un valor predeterminado
        return origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user

    except Exception as e:
        print(f"Error al procesar la línea: {log_line}")
        print(f"Error: {e}")

    return None
def process_log_file(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if 'GET /otp/plan' in line:
                date_str = re.search(r'\[([^:]+:[^ ]+)', line).group(1)
                
                # Convertir la fecha a un objeto datetime y añadir la zona horaria de La Paz
                date_time = datetime.strptime(date_str, '%d/%b/%Y:%H:%M:%S').replace(tzinfo=pytz.utc).astimezone(la_paz_timezone)

                route_info = extract_route_info(line)
                if route_info:
                    origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = route_info
                    
                    yield [date_time.strftime('%Y-%m-%d %H:%M:%S'),origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user]
                    
# Obtener la lista de archivos y ordenarlos según el prefijo numérico
file_pattern = re.compile(r'^\d{2}-')
files = os.listdir(dataset_logs)
log_files = [file for file in files if file_pattern.match(file)]
log_files.sort(key=lambda x: int(x.split('-')[0]))

# Guardar la información en un archivo CSV solo para rutas que cumplen con el patrón
csv_file_path = os.path.join(trufi_datos, 'origen-destino.csv')
header = ['date','origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude','userID']

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)

    for log_file in log_files:
        file_path = os.path.join(dataset_logs, log_file)
        
        route_info_generator = process_log_file(file_path)
        
        # Escribir las líneas en el archivo CSV
        csv_writer.writerows(route_info_generator)

print(f"Las rutas que cumplen con el patrón se han guardado en {csv_file_path}.")


Las rutas que cumplen con el patrón se han guardado en D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\origen-destino.csv.


## Descartar solicitudes sin user ID 

In [88]:
def filter_requests_with_userid(csv_file_path):
    # Leer el archivo CSV
    df = pd.read_csv(csv_file_path, parse_dates=['date'])

    # Contar filas antes de la filtración
    total_rows_before = len(df)

    # Filtrar las solicitudes que tienen userID
    df_filtered = df[df['userID'].notnull()]

    # Contar filas después de la filtración
    total_rows_after = len(df_filtered)

    # Calcular cuántas filas fueron eliminadas
    rows_removed = total_rows_before - total_rows_after

    # Sobrescribir el archivo original con las solicitudes filtradas
    df_filtered.to_csv(csv_file_path, index=False)

    return total_rows_before, rows_removed

# Llamar a la función para filtrar las solicitudes
total_rows_before, rows_removed = filter_requests_with_userid(csv_file_path)

# Imprimir la cantidad de filas eliminadas
print(f"Se eliminaron {rows_removed} filas sin userID de {total_rows_before}.")

Se eliminaron 83 filas sin userID de 1577937.


## Cantidad de usuarios a dentro del rango de fecha evaluado

In [89]:

# Obtener la cantidad de usuarios únicos
unique_users_count = df['userID'].nunique()

# Imprimir la cantidad de usuarios únicos
print(f"La cantidad de usuarios únicos es: {unique_users_count}")


La cantidad de usuarios únicos es: 99785


## Excluir puntos que esten dentro de lagos

In [90]:
df = pd.read_csv(csv_file_path, parse_dates=['date'])
df

,date,origin_latitude,origin_longitude,destination_latitude,destination_longitude,userID
0,2022-09-12 09:55:24,-17.399618,-66.161091,-17.382153,-66.165591,680c28ba-5008-4ffe-8026-3eaad4f832d1
1,2022-09-12 09:55:32,-17.379989,-66.167222,-17.390862,-66.159421,680c28ba-5008-4ffe-8026-3eaad4f832d1
2,2022-09-12 10:05:28,-17.390862,-66.159421,-17.379989,-66.167222,680c28ba-5008-4ffe-8026-3eaad4f832d1
3,2022-09-12 10:07:36,-17.373542,-66.165138,-17.383765,-66.159304,680c28ba-5008-4ffe-8026-3eaad4f832d1
4,2022-09-12 11:27:27,-17.383765,-66.159304,-17.373542,-66.165138,680c28ba-5008-4ffe-8026-3eaad4f832d1
...,...,...,...,...,...,...
1577849,2023-12-26 22:25:49,-17.388356,-66.197263,-17.377586,-66.058064,a4f158b9-1359-49a0-9d7f-d28f393727ee
1577850,2023-12-26 22:26:06,-17.418014,-66.160858,-17.401366,-66.182851,bb060471-bc27-4175-af11-301f0853bf33
1577851,2023-12-26 22:26:36,-17.412731,-66.159542,-17.383059,-66.159274,958041c4-c269-4603-9237-fb7b917f042b
1577852,2023-12-26 22:26:52,-17.412731,-66.159542,-17.369551,-66.174464,958041c4-c269-4603-9237-fb7b917f042b


In [106]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Crear puntos para las columnas de latitud y longitud
df['origin_point'] = df.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
df['destination_point'] = df.apply(lambda row: Point(row['destination_longitude'], row['destination_latitude']), axis=1)

# Crear un objeto multipolígono unión solo con los polígonos deseados
desired_objectIds = [3635, 3486, 3204, 2948]
print(f"Creados")
shapes_reader = shapefile.Reader(lagos_datos)
shapes = shapes_reader.shapes()
records = shapes_reader.records()

# Obtener información de las formas deseadas
desired_shapes = [shape(shape_).buffer(0) for i, shape_ in enumerate(shapes) if records[i][0] in desired_objectIds]

# Filtrar DataFrame para quedarse solo con los puntos que están fuera de los polígonos deseados
df_filtered = df[~df['origin_point'].apply(lambda point: any(point.within(shape_) for shape_ in desired_shapes)) & ~df['destination_point'].apply(lambda point: any(point.within(shape_) for shape_ in desired_shapes))]


Creados


In [107]:

# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

Solicitudes antes del filtrado: 1577854
Solicitudes después del filtrado: 1577111
Las solicitudes filtradas se han guardado en: D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\origen-destino.csv


## Excluir solicitudes de rutas con una distancia menor a 500 mts entre origen y destino

In [108]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Crear puntos para las columnas de latitud y longitud
df['origin_point'] = df.apply(lambda row: Point(row['origin_longitude'], row['origin_latitude']), axis=1)
df['destination_point'] = df.apply(lambda row: Point(row['destination_longitude'], row['destination_latitude']), axis=1)

# Calcular la distancia entre origen y destino en metros
df['distancia'] = df.apply(lambda row: distance((row['origin_latitude'], row['origin_longitude']),
                                               (row['destination_latitude'], row['destination_longitude'])).meters, axis=1)

# Filtrar DataFrame para quedarse solo con los puntos que tienen una distancia mayor a 500 metros
df_filtered = df[df['distancia'] > 500]

In [109]:
# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV sin las columnas origin_point y destination_point
df_filtered.drop(['origin_point', 'destination_point'], axis=1, inplace=True)
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

Solicitudes antes del filtrado: 1577111
Solicitudes después del filtrado: 1446500
Las solicitudes filtradas se han guardado en: D:\Trufiapp\GANS\Anexos\Datos\Registros de Trufi App\origen-destino.csv


## Generar variables para municipios origen y destino

In [ ]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Leer el archivo GeoJSON
with open(municipios_datos) as f:
    municipios_geojson = json.load(f)

def get_city_from_coords(coords):
    point = Point(coords)
    # Iterar sobre las características del GeoJSON
    for feature in municipios_geojson['features']:
        municipio_geometry = shape(feature['geometry'])
        # Verificar si el punto está dentro de la geometría del municipio
        if point.within(municipio_geometry):
            return feature['properties']['CAPITAL']
    
    return 'externo'

# Ahora puedes usar esta función para asignar nombres de municipios a tus datos
df['origin_municipio'] = df.apply(lambda row: get_city_from_coords((row['origin_longitude'], row['origin_latitude'])), axis=1)
df['dest_municipio'] = df.apply(lambda row: get_city_from_coords((row['destination_longitude'], row['destination_latitude'])), axis=1)

# Mostrar el DataFrame con los nombres de los municipios
print(df)

In [ ]:
# Guardar el DataFrame filtrado en un nuevo archivo CSV
df.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

## Excluir solicitudes de rutas fuera de Cochabamba

In [ ]:
# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'])

# Filtrar el DataFrame para excluir las solicitudes con origen o destino "externo"
df_filtered = df[(df['origin_municipio'] != 'externo') | (df['dest_municipio'] != 'externo')]

# Contar líneas antes del filtrado
rows_before = len(df)

# Contar líneas después del filtrado
rows_after = len(df_filtered)

# Imprimir la cantidad de líneas antes y después del filtrado
print(f"Solicitudes antes del filtrado: {rows_before}")
print(f"Solicitudes después del filtrado: {rows_after}")

# Guardar el DataFrame filtrado en un nuevo archivo CSV
df_filtered.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

## Generar datos sinteticos

In [ ]:
# Agregar características de tiempo y distancia
df['hora'] = df['date'].dt.hour
df['dia_de_semana'] = df['date'].dt.dayofweek  # Lunes: 0, Domingo: 6
df['dia_de_mes'] = df['date'].dt.day
df['fin_de_semana'] = (df['date'].dt.weekday >= 5).astype(int)  # 1 si es fin de semana, 0 si no

# Eliminar la columna 'date' si ya no es necesaria
df = df.drop('date', axis=1)

# Mostrar el DataFrame resultante
print(df)


# Guardar el DataFrame filtrado en un nuevo archivo CSV
df.to_csv(csv_file_path, index=False)

print(f"Las solicitudes filtradas se han guardado en: {csv_file_path}")

## Integrar poligonos h3

In [ ]:
# Función para obtener el índice H3 y las coordenadas
def get_h3_index_and_coords(latitude, longitude, resolution=7):
    h3_index = h3.geo_to_h3(latitude, longitude, resolution)
    coords = h3.h3_to_geo(h3_index)
    return h3_index, coords

# Leer el archivo CSV en un DataFrame
df = pd.read_csv(csv_file_path, parse_dates=['date'], encoding='latin1')

# Crear columnas para índices H3 de origen y destino
df['origin_h3_index'], df['origin_coords'] = zip(*df.apply(lambda row: get_h3_index_and_coords(row['origin_latitude'], row['origin_longitude']), axis=1))
df['dest_h3_index'], df['dest_coords'] = zip(*df.apply(lambda row: get_h3_index_and_coords(row['destination_latitude'], row['destination_longitude']), axis=1))

# Crear un diccionario que mapea cada índice H3 único a su propio ID único
h3_index_to_id = {h3_index: idx + 1 for idx, h3_index in enumerate(pd.concat([df['origin_h3_index'], df['dest_h3_index']]).unique())}

# Crear DataFrame con índices H3 únicos y sus IDs correspondientes y coordenadas
h3_id_df = pd.DataFrame(list(h3_index_to_id.items()), columns=['h3_index', 'id'])
h3_id_df[['latitude', 'longitude']] = pd.DataFrame(h3_id_df['h3_index'].apply(lambda h3_index: h3.h3_to_geo(h3_index)).to_list(), index=h3_id_df.index)

# Guardar DataFrame de índices H3 y sus IDs en un archivo CSV separado
h3_id_df.to_csv(h3_datos, index=False)

# Asignar los IDs únicos tanto para el origen como para el destino utilizando el diccionario
df['origin_id'] = df['origin_h3_index'].map(h3_index_to_id)
df['destination_id'] = df['dest_h3_index'].map(h3_index_to_id)

# Crear un nuevo DataFrame solo con las columnas necesarias
new_df = df[['date', 'hora', 'dia_de_semana', 'dia_de_mes', 'fin_de_semana', 'distancia', 'origin_id', 'destination_id']]

# Guardar el nuevo DataFrame con IDs únicos y solo registros en Cochabamba en un nuevo directorio
os.makedirs(output_dir, exist_ok=True)
new_df.to_csv(output_file_path, index=False)

print(f"La información modificada se ha guardado en el archivo {output_file_path}.")
max_origin_id = df['origin_id'].max()
max_destination_id = df['destination_id'].max()

# Obtener una lista de IDs únicos correspondientes a origin_id
unique_origin_ids = df['origin_id'].unique().tolist()

# Imprimir la lista de IDs únicos
print("Lista de IDs únicos de origin_id:")
print(unique_origin_ids)
print(f"El máximo ID de destinos en Cochabamba es: {max_destination_id}")

In [ ]:
# Load the DataFrame with H3 indices and coordinates
h3_id_df = pd.read_csv(h3_datos)

# Calculate the average latitude and longitude
avg_lat = h3_id_df['latitude'].mean()
avg_lon = h3_id_df['longitude'].mean()

# Create a map centered at the average coordinates
m = folium.Map(location=[avg_lat, avg_lon], zoom_start=12)

# Set to keep track of printed hexagons
printed_hexagons = set()

h3_id_df.head()

In [ ]:

# Function to add a polygon to the map with a label
def add_polygon_with_label(m, hexagon, label):
    folium.Polygon(
        locations=hexagon,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.4,
        tooltip=f'Label: {label}'  # Add tooltip with the label number
    ).add_to(m)
    printed_hexagons.add(label)  # Use 'label' as a unique identifier

# Iterate over the records
for idx, row in h3_id_df.iterrows():
    # Get the coordinates and convert them to a tuple
    coords = (row['latitude'], row['longitude'])
    hexagon_coords = h3.h3_to_geo_boundary(row['h3_index'])  # Get hexagon coordinates

    # Check if the hexagon has not been printed yet
    if row['id'] not in printed_hexagons:  # Use 'id' as a unique identifier
        add_polygon_with_label(m, hexagon_coords, row['id'])  # Use 'id' as a label

# Save the map of origins and destinations in Cochabamba as an HTML file
map_cochabamba_file_path = os.path.join(repo, 'h3_map_cochabamba_.html')
m.save(map_cochabamba_file_path)

In [ ]:
# Imprimir los hexágonos impresos
print("Hexágonos impresos:", len(printed_hexagons))

## Normalizar fuentes de datos